In [2]:
function sig(x)
    return 1 / (1 + exp(-x))
end

sig (generic function with 1 method)

In [3]:
function f(x1, w)
    # 決定境界の直線の方程式
    # p(C1|X) > 0.5 -> XをC1に分類
    # p(C1|X) < 0.5 -> XをC2に分類
    # 次元は全部同じなので，"./"などは不要
    a = - (w[2] / w[3])
    b = - (w[1] / w[3])
    return a * x1 + b
end

f (generic function with 1 method)

In [4]:
using Distributions
using Plots
gr()
ENV["PLOTS_TEST"] = "true"

# 訓練データを作成
N = 500
mean1 = [-2 2]
mean2 = [1 -1]
mean3 = [8 -6]
cov = [1.0 0.8; 0.8 1.0]

# ノイズありデータ作成
cls1 = zeros(0)
for i in 1:2
    d = Normal(mean1[i], cov[i]) # mean1とcovから生成した正規分布
    append!(cls1, rand(d, div(N, 2))) # 正規分布からランダムにN/2個選択
end
cls1 = reshape(cls1, (div(N, 2), 2))

cls2 = zeros(0)
for i in 1:2
    d = Normal(mean2[i], cov[i, 2])
    append!(cls2, rand(d, div(N, 2) - div(N, 5)))
    d = Normal(mean3[i], cov[i, 2])
    append!(cls2, rand(d, div(N, 5)))
end
cls2 = reshape(cls2, (div(N, 2), 2))

# Xの作成
cls = vcat(cls1, cls2)
temp = ones(N)
X = hcat(temp, cls)

# ラベルTを作成
t = zeros(0)
for i in 1:div(N, 2)
    append!(t, 1.0)
end
for i in 1:div(N, 2)
    append!(t, 0.0)
end

# パラメータwをIRLSで更新
turn = 0
w = [0.0 0.0 0.0]  # 適当な初期値
while true
    phi = X

    # Rとyを計算
    R = zeros(N, N)
    y = zeros(0)
    for n in 1:N
        a = dot(w, phi[n, :])
        y_n = sig(a)
        R[n, n] = y_n * (1.0 - y_n)
        append!(y, y_n)
    end
    
    # ヘッセ行列Hを計算
    phi_T = phi' #3*100
    H = phi_T * R * phi

    # wを更新
    w_new = vec(w) .- inv(H) * (phi_T * (y - t))

    # wの収束判定
    diff = norm(w_new .- w) ./ norm(w)
    println("turn:", turn)
    println("diff:", diff)
    if diff < 0.05
        break
    end
    
    w = w_new
    turn += 1
end

turn:0
diff:Inf
turn:1
diff:0.7190433148157277
turn:2
diff:0.7864995794039409
turn:3
diff:0.5392009983751288
turn:4
diff:0.38554659220118176
turn:5
diff:0.3250083511905655
turn:6
diff:0.29862131978970974
turn:7
diff:0.27184062282614674
turn:8
diff:0.2126659433949613
turn:9
diff:0.11294473561849873
turn:10
diff:0.026920059821388728


In [5]:
#入力データの描画
x = cls[:, 1]
y = cls[:, 2]
x = reshape(x, (div(N, 2), 2))
y = reshape(y, (div(N, 2), 2))
plot(x, y, seriestype=:scatter)

# 識別境界を描画
x1 = linspace(-6, 10, 100)
x2 = [f(x, w) for x in x1]
plot!(x1, x2) # "!"をつけることでレイヤーの上書きをする
# savefig("logistic.png")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 -7.5 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip7402)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 54.808,275.85 59.9616,273.287 65.1151,270.724 70.2687,268.161 75.4222,265.598 80.5758,263.035 85.7293,260.472 90.8829,257.909 96.0364,255.346 101.19,252.783 
 106.343,250.22 111.497,247.657 116.651,245.094 121.804,242.531 126.958,239.968 132.111,237.405 137.265,234.842 142.418,232.279 147.572,229.716 152.725,227.153 
 157.879,224.59 163.032,222.027 168.186,219.464 173.34,216.901 178.493,214.338 183.647,211.775 188.8,209.212 193.954,206.649 199.107,204.086 204.261,201.523 
 209.414,198.96 214.568,196.397 219.721,193.834 224.875,191.271 230.029,188.708 235.182,186.145 240.336,183.582 245.489,181.019 250.643,178.456 255.796,175.894 
 260.95,173.331 266.103,170.768 271.257,168.205 276.41,165.642 281.564,163.079 286.718,160.516 291.871,157.953 297.025,155.39 302.178,152.827 307.332,150.264 
 312.485,147.701 317.639,145.138 322.792,142.575 327.946,140.012 333.099,137.449 338.253,134.886 343.407,132.323 348.56,129.76 353.714,127.197 358.867,124.634 
 364.021,122.071 369.174,119.508 374.328,116.945 379.481,114.382 384.635,111.819 389.788,109.256 394.942,106.693 400.096,104.13 405.249,101.567 410.403,99.004 
 415.556,96.441 420.71,93.8781 425.863,91.3151 431.017,88.7521 436.17,86.1891 441.324,83.6261 446.477,81.0631 451.631,78.5002 456.785,75.9372 461.938,73.3742 
 467.092,70.8112 472.245,68.2482 477.399,65.6852 482.552,63.1223 487.706,60.5593 492.859,57.9963 498.013,55.4333 503.166,52.8703 508.32,50.3074 513.473,47.7444 
 518.627,45.1814 523.781,42.6184 528.934,40.0554 534.088,37.4924 539.241,34.9295 544.395,32.3665 549.548,29.8035 554.702,27.2405 559.855,24.6775 565.009,22.1145 
 
 "/>
 
 
 
 
 
 y1 
 
 
 
 
 y2 
 
 
 
 y3